# COVID-19 Data Analysis in Alberta
*Prepared by Willem Klumpenhouwer*

This workbook goes through a number of possible data visualizations/aggregations of detailed case information on the COVID-19 pandemic in Alberta, Canada.

**Note: This workbook comes with no guarantee of accuracy, beyond the general transparancy of the code and the data behind it. The author makes no promises about the quality of the visualizations, the reception of the content, or any conclusions drawn based on the data**

## Initializations and Data Setup
We start by importing data directly from a Google Sheet maintained by Robson Fletcher of CBC, followed by some data cleaning and the assignment of date types to the date column. We'll also put all our style definitions in here so that we can keep charts a consistent size (for Tweeting, for example). For reference, color schemes can be found:

https://vega.github.io/vega/docs/schemes/

In [172]:
import pandas as pd
import altair as alt
from datetime import date, timedelta
import os
width = 600
height = 400
font = 'Raleway'
labelColor='#A9A9A9'
color = {
    'red': '#E70300',
    'orange': '#F58426'
}
data_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSWRP25zy0xC2m7_pwz5qvkUToMlY8on1c33DByQ5aduaPY3oTO4b4mLDI6z-e01P64oWZPD-vJY6zw/pub?gid=0&single=true&output=csv'
raw = pd.read_csv(data_url)
raw['date'] = pd.to_datetime(raw['date'])
raw.sort_values('date', inplace=True)
raw.head()

,id,date,zone,sex,age_range,status,confirmation
0,1,2020-03-06,Calgary Zone,Female,50-59 years,Recovered,Confirmed
1,2,2020-03-09,Calgary Zone,Female,30-39 years,Recovered,Confirmed
2,3,2020-03-09,Calgary Zone,Male,30-39 years,Recovered,Confirmed
3,4,2020-03-09,Edmonton Zone,Male,60-69 years,Recovered,Confirmed
4,5,2020-03-09,Edmonton Zone,Male,40-49 years,Recovered,Confirmed


## Getting Started: New Cases
We'll consider only confirmed cases for this. For a total province-wide count, we'll group them by date, and count the total number. Then, to count the number of new cases, we'll `shift` the case count column by one and calculate a difference. This difference will have to be divided by the total days passed.

In [184]:
confirmed = raw[raw['confirmation'] == 'Confirmed'].copy()
confirmed = confirmed[['id', 'date']].groupby('date', as_index=False).count()
confirmed.columns = ['date', 'reported']  # Rename the columns for ease of use
confirmed['date_next'] = confirmed['date'].shift(-1)
confirmed.dropna(inplace=True)  # Removes the shifted row at the bottom of the tabl
confirmed['new_daily'] = confirmed['reported']/(confirmed['date_next']-confirmed['date']).dt.days
confirmed.head()

,date,reported,date_next,new_daily
0,2020-03-06,1,2020-03-09,0.333333
1,2020-03-09,6,2020-03-10,6.000000
2,2020-03-10,9,2020-03-11,9.000000
3,2020-03-11,7,2020-03-12,7.000000
4,2020-03-12,2,2020-03-13,2.000000


Next, the plot of new cases. We'll do a rolling average line (calculated automagically by the plotting software) with the points in black to show the overall data. This is done by layering four elements: The `line` plot with the rolling average (7 days previous), the `points` plot which shows the scatterplot, the `pointLabel` which puts a label at the value of the maximum date (with some offset), and the `lineLabel` which puts the line value at the maixmum date. We finish up by layering the plots (adding them in the order of rendering) and then customizing the look/feel of the plot a bit.

In [189]:
line = alt.Chart(total_active[['new_daily', 'date']]).mark_circle(
    size=15, color='black'
).encode(
    x=alt.X('date:T', axis=alt.Axis(format='%b', grid=False, tickCount=6), title=None),
    y=alt.Y('new_daily:Q', title=None)
)

points = alt.Chart(total_active[['new_daily', 'date']]).mark_line(
    color=color['orange'],
    size=3
).transform_window(
    rolling_mean='mean(new_daily)',
    frame=[-7,0]
).encode(
    x=alt.X('date:T'),
    y=alt.Y('rolling_mean:Q')
)

pointLabel = alt.Chart(total_active[['new_daily', 'date']]).mark_text(
    align='left', 
    dx=5,
    fontWeight='bold'
).encode(
    alt.X('date:T', aggregate='max', title=None),
    alt.Y('new_daily:Q', aggregate={'argmax': 'date'}, title=None),
    alt.Text("new_daily:Q", aggregate={'argmax': 'date'}, title=None)
)

lineLabel = alt.Chart(total_active[['new_daily', 'date']]).mark_text(
    align='left', 
    dx=5,
    color=color['orange'],
    fontWeight='bold'
).transform_window(
    rolling_mean='mean(new_daily)',
    frame=[-7,0]
).encode(
    alt.X('date:T', aggregate='max', title=None),
    alt.Y('rolling_mean:Q', aggregate={'argmax': 'date'}, title=None),
    alt.Text("rolling_mean:Q", aggregate={'argmax': 'date'}, title=None, format=',.2r')
)

(line+points+pointLabel+lineLabel).properties(
    width=width, 
    height=height,
    title="New COVID-19 cases in Alberta, daily",
).configure_title(
    fontSize=20,
    anchor='start',
    font=font
).configure_axis(
    labelFontSize=14,
    labelColor=labelColor,
    labelFontWeight = 'bold'
)

alt.LayerChart(...)

### Case Acceleration
One thing to ask the data is *how are the number of new cases changing over time?*. If new cases are the "speed" at which the pandemic is travelling through the population, then if we take the change in speed over time (the slope of the new cases graph at a given point) we can see whether the rate of spread is growing or remaining constant.

To do this, we'll need to do another shift on the above dataset, and again normalize over time. We are, after all, taking the second derivative of the total cases. I'd like to start with a fresh copy of the data so we avoid repeatedly shifting/updating our existing

In [190]:
accel = confirmed.copy()
accel['new_next'] = accel['reported'].shift(-1)
accel['new_date_next'] = accel['date_next'].shift(-1)
accel['accel'] = (accel['new_next'] - accel['reported'])/(accel['new_date_next'] - accel['date_next']).dt.days # Same process as above but combined into one line
accel.head()

,date,reported,date_next,new_daily,new_next,new_date_next,accel
0,2020-03-06,1,2020-03-09,0.333333,6.0,2020-03-10,5.0
1,2020-03-09,6,2020-03-10,6.000000,9.0,2020-03-11,3.0
2,2020-03-10,9,2020-03-11,9.000000,7.0,2020-03-12,-2.0
3,2020-03-11,7,2020-03-12,7.000000,2.0,2020-03-13,-5.0
4,2020-03-12,2,2020-03-13,2.000000,8.0,2020-03-14,6.0


To show a bit more information at once, we are going to use the `mark_trail()` feature to show new cases (the speed) by the thickness of the lines. Again, we'll use a rolling average (7 previous days) to keep things smooth, and put some underlying data below it.

In [191]:
chart = alt.Chart(accel[['date', 'new_daily', 'accel']]).mark_trail(color=color['orange']
).transform_window(
    rolling_mean='mean(accel)',
    frame=[-7,7]
).encode(
    x=alt.X('date:T', axis=alt.Axis(format = ("%b %d"), title=None, grid=False)),
    y=alt.Y('accel:Q', title=None),
    size=alt.Size(
        'new_daily:Q', 
        scale=alt.Scale(range=[0,4]), 
        title='New Daily Cases', 
        legend=alt.Legend(orient="bottom")
    )
).properties(
    width=600, 
    height=200,
    title="Change in new COVID-19 cases in Alberta, daily",
).configure_title(
    fontSize=20,
    anchor='start',
    font=font
).configure_axis(
    labelFontSize=12,
    labelColor=labelColor
)
chart

alt.Chart(...)

## Active Cases
Let's have a look now only at *active* cases, and see what we can tease out of the data.

Let's start by looking at recent and active cases. We'll filter out cases older than 3 weeks, and ones that aren't active.

In [192]:
three_weeks_ago = date.today() - timedelta(weeks=3)
active = raw[(raw['status'] == 'Active') & (raw['date'].dt.date >= three_weeks_ago)].copy()
active.head()

,id,date,zone,sex,age_range,status,confirmation
23445,23446,2020-10-21,Calgary Zone,Male,70-79 years,Active,Confirmed
23250,23251,2020-10-21,Calgary Zone,Female,80+ years,Active,Confirmed
23957,23958,2020-10-22,Edmonton Zone,Male,80+ years,Active,Confirmed
23750,23751,2020-10-22,Edmonton Zone,Female,70-79 years,Active,Confirmed
23735,23736,2020-10-22,Calgary Zone,Male,30-39 years,Active,Confirmed


It might be nice to get a set of sparkline plots to show the case profiles by region for comparison.

In [211]:
regional = active[['date', 'id', 'zone']].groupby(['date', 'zone'], as_index=False).count()
regional.columns = ['date', 'zone', 'reported']
regional['zone'] = regional['zone'].str.replace(' Zone', '')
regional.head()

,date,zone,reported
0,2020-10-21,Calgary,2
1,2020-10-22,Calgary,2
2,2020-10-22,Edmonton,2
3,2020-10-23,Calgary,2
4,2020-10-23,Central,1


In [225]:
alt.Chart(regional).mark_line(
    color=color['orange']
).transform_window(
    rolling_mean='mean(reported)',
    frame=[-7,0]
).encode(
    x=alt.X('date:T', axis=alt.Axis(format = ("%b %d"), title=None, grid=False)),
    y=alt.Y('rolling_mean:Q', axis=alt.Axis(tickCount=2, title=None)),
    row=alt.Row('zone', title="Zone")
).properties(
    width=600, 
    height=40,
    title="New COVID-19 Cases in past 3 weeks by Region, daily"
)

alt.Chart(...)